# Redes neurais artificiais

In [2]:
import h5py
import torch
import torch.nn as nn
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from skorch import NeuralNetBinaryClassifier, NeuralNetClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

## Redes MPL

In [3]:
data = pd.read_csv('data/credtrain.txt', sep='\t', header=None)
data_test = pd.read_csv('data/credtest.txt', sep='\t', header=None)

X = np.array(data.iloc[:, :-1]).astype(np.float32)
y = np.array(data.iloc[:, -1]).astype(np.float32)

X_test = np.array(data_test.iloc[:, :-1]).astype(np.float32)
y_test = np.array(data_test.iloc[:, -1])

ss = StandardScaler()
X_norm = ss.fit_transform(X)
X_test_norm = ss.fit_transform(X_test)

In [6]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(11, 64)
        self.fc2 = torch.nn.Linear(64, 1)
        
    def forward(self, x, **kwargs):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = NeuralNetBinaryClassifier(Net, max_epochs=200, lr=0.01, criterion=torch.nn.BCEWithLogitsLoss, verbose=0)
net.fit(X_norm, y)
y_predict = net.predict(X_test_norm)
print(classification_report(y_predict, y_test))

              precision    recall  f1-score   support

           0       0.98      0.85      0.91       352
           1       0.80      0.97      0.88       225

    accuracy                           0.90       577
   macro avg       0.89      0.91      0.89       577
weighted avg       0.91      0.90      0.90       577



## Redes convolucionais

### MPL

In [10]:
def load_dataset():
    train_dataset = h5py.File('data/train_catvnoncat.h5', 'r' )
    train_set_x_orig = np.array(train_dataset['train_set_x'][:]).astype(np.float32) #your train set features
    train_set_y_orig = np.array(train_dataset['train_set_y'][:]) #your train set labels

    test_dataset = h5py.File('data/test_catvnoncat.h5', 'r' )
    test_set_x_orig = np.array(test_dataset['test_set_x'][:]).astype(np.float32) #your test set features
    test_set_y_orig = np.array(test_dataset['test_set_y'][:]) #your test set labels

    classes = np.array(test_dataset['list_classes'][:]) #the list of classes

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

X_train, y_train, X_test, y_test, classes = load_dataset()

X_train_flatten = X_train.reshape(X_train.shape[0], -1) / 255
X_test_flatten = X_test.reshape(X_test.shape[0], -1) / 255

In [83]:
class NetSoftmax(torch.nn.Module):
    def __init__(self):
        super(NetSoftmax, self).__init__()
        self.fc1 = torch.nn.Linear(64 * 64 * 3, 64)
        self.fc2 = torch.nn.Linear(64, 2)
        
    def forward(self, x, **kwargs):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
net_softmax = NeuralNetClassifier(NetSoftmax, max_epochs=250, lr=0.01, criterion=torch.nn.CrossEntropyLoss, verbose=0)
net_softmax.fit(X_train_flatten, y_train)

y_predict = net_softmax.predict(X_test_flatten)

print(classification_report(y_predict, y_test))

              precision    recall  f1-score   support

           0       0.65      0.52      0.58        21
           1       0.70      0.79      0.74        29

    accuracy                           0.68        50
   macro avg       0.67      0.66      0.66        50
weighted avg       0.68      0.68      0.67        50



In [167]:
import torchvision.transforms as T

class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.transform = T.Compose([
            T.RandomHorizontalFlip(p=0.5),
            T.ToTensor(),
            T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X = self.X[idx]
        y = self.y[idx]
        X = Image.fromarray(np.uint8(X)).convert('RGB')
        X = self.transform(X)
        return X, y

dataset = CustomDataset(X_train, y_train)
dataset_test = CustomDataset(X_test, y_test)

In [135]:
class SimpleCNN(torch.nn.Module):
    
    #Our batch shape for input x is (3, 32, 32)
    
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        #Input channels = 3, output channels = 18
        self.conv1 = torch.nn.Conv2d(3, 18, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        #4608 input features, 64 output features (see sizing flow below)
        self.fc1 = torch.nn.Linear(18 * 32 * 32, 64)
        
        #64 input features, 10 output features for our 10 defined classes
        self.fc2 = torch.nn.Linear(64, 2)
        
    def forward(self, x):
        
        #Computes the activation of the first convolution
        #Size changes from (3, 32, 32) to (18, 32, 32)
        x = F.relu(self.conv1(x))
        
        #Size changes from (18, 32, 32) to (18, 16, 16)
        x = self.pool(x)
        
        #Reshape data to input to the input layer of the neural net
        #Size changes from (18, 16, 16) to (1, 4608)
        #Recall that the -1 infers this dimension from the other given dimension
        x = x.view(-1, 18 * 32 * 32)
        
        #Computes the activation of the first fully connected layer
        #Size changes from (1, 4608) to (1, 64)
        x = F.relu(self.fc1(x))
         
        #Computes the second fully connected layer (activation applied later)
        #Size changes from (1, 64) to (1, 10)
        x = self.fc2(x)
        return(x)

In [171]:
net_cnn = NeuralNetClassifier(
    SimpleCNN,
    max_epochs=50,
    lr=0.001,
    criterion=torch.nn.CrossEntropyLoss,
    optimizer=torch.optim.Adam,
    train_split=None,
    verbose=1
)

net_cnn.fit(dataset, y_train)
y_predict = net_cnn.predict(dataset_test)
print(classification_report(y_predict, y_test))

  epoch    train_loss     dur
-------  ------------  ------
      1        1.6877  0.6990
      2        0.9874  0.8366
      3        0.7514  0.9216
      4        0.6531  0.8181
      5        0.5872  0.8253
      6        0.6048  0.8675
      7        0.5970  0.8285
      8        0.5610  0.8083
      9        0.5316  0.9484
     10        0.5272  1.0702
     11        0.5308  1.1411
     12        0.5117  1.2366
     13        0.4906  0.8384
     14        0.4780  0.9252
     15        0.4708  1.0266
     16        0.4553  0.9583
     17        0.4367  0.7969
     18        0.4241  0.7519
     19        0.4144  1.0092
     20        0.4012  1.0828
     21        0.3878  1.0565
     22        0.3756  0.8025
     23        0.3626  0.9251
     24        0.3517  1.3072
     25        0.3451  1.2478
     26        0.3334  1.1201
     27        0.3218  1.0272
     28        0.3124  0.9361
     29        0.3014  0.9116
     30        0.2922  0.8590
     31        0.2838  0.8176
     32   